In [ ]:
!pip install GPBoost -U 
import os
import csv
from google.colab import drive 
drive.mount('/content/drive')
my_path = "/content/drive/My Drive/Thesis/simulations_new/sparse_data_setting"
os.chdir(my_path)

from simulation_utils import * 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import gpboost as gpb
#import pdpbox
#from pdpbox import pdp
sns.set()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 5.2 MB/s 
Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Initial experiments - Small data setting (only 10 observations per group)

## Linear function; Shared GP in data generating process

In [ ]:
# Generate data
n, m = 500, 50  # Number of observations and groups
p = int(n/m) # Number of observations per group
n_datasets = 20 
n_valid = 5
datasets, validation_datasets = generate_datasets(n, m, p, n_datasets, n_valid, func='linear_func', random_state=60, shared_gp=True)

# Create groups
groups = np.arange(n)
for i in range(m):
    groups[i*p:(i+1)*p] = i

# Create times for validation datasets
times = np.tile(np.arange(p)*10+10, m)

## GPBoost hyperparameter tuning

### Random Intercept model

In [ ]:
# Random slope
gp_model = gpb.GPModel(group_data=np.column_stack((groups, times)))
# Candidate parameter grid
param_grid = {'learning_rate': [0.01, 0.1, 0.5, 1], 
              'max_depth': [1, 2, 3, 4, 5, 10, 50],
              'min_data_in_leaf': [1, 10, 20, 50]}

# Other parameters not contained in the grid of tuning parameters
params = {'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

opt_params = grid_search_tune_parameters_multiple(param_grid=param_grid,
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=gp_model,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets['data_train'],
                                                  verbose_eval=0,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')

print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 112 parameter combinations...


 |--------------------------------------------------| 0.8929% 



 |█-------------------------------------------------| 1.7857% 



 |█-------------------------------------------------| 2.6786% 



 |██------------------------------------------------| 3.5714% 



 |██------------------------------------------------| 4.4643% 



 |███-----------------------------------------------| 5.3571% 



 |███-----------------------------------------------| 6.2500% 



 |████----------------------------------------------| 7.1429% 



 |████----------------------------------------------| 8.0357% 



 |████----------------------------------------------| 8.9286% 



 |█████---------------------------------------------| 9.8214% 



 |█████---------------------------------------------| 10.7143% 



 |██████--------------------------------------------| 11.6071% 



 |██████--------------------------------------------| 12.5000% 



 

### GPBoost with independent GPs

In [ ]:
# Independent GP
gp_model = gpb.GPModel(group_data=groups, gp_coords=times, cluster_ids=groups, cov_function='exponential', cov_fct_shape=2.5)

# Candidate parameter grid
param_grid = {'learning_rate': [0.01, 0.1, 0.5, 1], 
              'max_depth': [1, 2, 3, 4, 5, 10, 50],
              'min_data_in_leaf': [1, 10, 20, 50]}

# Other parameters not contained in the grid of tuning parameters
params = {'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

opt_params = grid_search_tune_parameters_multiple(param_grid=param_grid,
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=gp_model,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets['data_train'],
                                                  verbose_eval=1,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 112 parameter combinations...


 |--------------------------------------------------| 0.8929% 

Trying parameter combination 1 of 112: {'learning_rate': 0.01, 'max_depth': 1, 'min_data_in_leaf': 1} ...
***** New best score (1.8350213047078676) found for the following parameter combination:
{'learning_rate': 0.01, 'max_depth': 1, 'min_data_in_leaf': 1, 'num_boost_round': 999.0}


 |█-------------------------------------------------| 1.7857% 

Trying parameter combination 2 of 112: {'learning_rate': 0.1, 'max_depth': 1, 'min_data_in_leaf': 1} ...
***** New best score (1.7859072087549848) found for the following parameter combination:
{'learning_rate': 0.1, 'max_depth': 1, 'min_data_in_leaf': 1, 'num_boost_round': 255.4}


 |█-------------------------------------------------| 2.6786% 

Trying parameter combination 3 of 112: {'learning_rate': 0.5, 'max_depth': 1, 'min_data_in_leaf': 1} ...
***** New best score (1.7838973663871371) found for the follo

### GPBoost with shared GPs

In [ ]:
# Shared GP
gp_model = gpb.GPModel(group_data=groups, gp_coords=times, cov_function='exponential', cov_fct_shape=2.5)

# Use the same hyperparameters as for Independent GP and only cross-validate to find num_boost_round
params = {'learning_rate': 0.5,
          'max_depth': 1,
          'min_data_in_leaf': 50,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

opt_params = grid_search_tune_parameters_multiple(param_grid={},
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=gp_model,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets['data_train'],
                                                  verbose_eval=1,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 1 parameter combinations...


 |██████████████████████████████████████████████████| 100.0000% 


Trying parameter combination 1 of 1: {} ...
***** New best score (1.561520143422572) found for the following parameter combination:
{'num_boost_round': 46.8}
Best number of iterations:  46.8
Best score:  1.561520143422572
Best parameters:  {}


### Gradient-boosted trees including groups as a categorical variable

In [ ]:
# GBT with grouping variable as a categorical feature
datasets_cat, validation_datasets_cat = generate_datasets(n, m, p, n_datasets, n_valid, func='linear_func', 
                                                            shared_gp=True, random_state=60, include_cat_feature=True)

# Candidate parameter grid
param_grid = {'num_leaves': [10, 20, 50, 200, 500, 1000, 5000, 2**10], 
              'max_depth': [1, 2, 3, 4, 5, 10, 50],
              'min_data_in_leaf': [1, 5, 10, 20, 30, 50]}

# Other parameters not contained in the grid of tuning parameters
params = {'objective': 'regression_l2',
          'verbose': 0,
          'learning_rate' : 0.10}
          #'device_type' : 'gpu'}

opt_params = grid_search_tune_parameters_multiple(param_grid=param_grid,
                                                  params=params,
                                                  nfold=5,
                                                  gp_model=None,
                                                  use_gp_model_for_validation=True,
                                                  train_sets=validation_datasets_cat['data_train'],
                                                  verbose_eval=1,
                                                  num_boost_round=1000, 
                                                  early_stopping_rounds=10,
                                                  shuffle=False,
                                                  seed=1,
                                                  metrics='rmse')
                                                  
print('Best number of iterations: ', opt_params['best_iter'])
print('Best score: ', opt_params['best_score'])
print('Best parameters: ', opt_params['best_params'])

Starting deterministic grid search with 336 parameter combinations...


 |--------------------------------------------------| 0.2976% 

Trying parameter combination 1 of 336: {'num_leaves': 10, 'max_depth': 1, 'min_data_in_leaf': 1} ...


/usr/local/lib/python3.7/dist-packages/gpboost/basic.py:1822: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/gpboost/basic.py:1339: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


***** New best score (1.5984542243939583) found for the following parameter combination:
{'num_leaves': 10, 'max_depth': 1, 'min_data_in_leaf': 1, 'num_boost_round': 220.8}


 |--------------------------------------------------| 0.5952% 

Trying parameter combination 2 of 336: {'num_leaves': 20, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |--------------------------------------------------| 0.8929% 

Trying parameter combination 3 of 336: {'num_leaves': 50, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |█-------------------------------------------------| 1.1905% 

Trying parameter combination 4 of 336: {'num_leaves': 200, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |█-------------------------------------------------| 1.4881% 

Trying parameter combination 5 of 336: {'num_leaves': 500, 'max_depth': 1, 'min_data_in_leaf': 1} ...


 |█-------------------------------------------------| 1.7857% 

Trying parameter combination 6 of 336: {'num_leaves': 1000, 'max_depth': 1, 'min_data_in_le

In [ ]:
# Create results dataframe
results_1 = pd.DataFrame(columns=["RMSE Interpolation (mean)", "RMSE Interpolation (std)", "RMSE Extrapolation (mean)", "RMSE Extrapolation (std)", "RMSE_F (mean)", "RMSE_F (std)", "Time"],
                         index=['Linear Mixed Effects Model with no fixed features (random intercept)',
                                   'Linear Mixed Effects Model with Random Intercept',
                                   'Linear Mixed Effects Model with Shared Gaussian Process',
                                   'Linear Mixed Effects Model with Independent Gaussian Process',
                                   'Gradient-boosted tree with group as categorical variable (no random effects)',
                                   'GPBoost with Random Intercept',
                                   'GPBoost with Shared Gaussian Process',
                                   'GPBoost with Independent Gaussian Process'])

In [ ]:
# 1. Linear Mixed Effects Model with no fixed features (random intercept)
time_list, RMSE_list1, RMSE_list2 = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=False, no_features=True)

results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "Time"] = np.mean(time_list)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["Linear Mixed Effects Model with no fixed features (random intercept)", "RMSE Interpolation (std)"] = np.std(RMSE_list2)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# 2. Linear Mixed Effects Model with Random Intercept
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=False)

results_1.loc["Linear Mixed Effects Model with Random Intercept", "Time"] = np.mean(time_list)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["Linear Mixed Effects Model with Random Intercept", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# 3. Linear Mixed Effects Model with Shared Gaussian Process
mod_name = 'Linear Mixed Effects Model with Shared Gaussian Process'
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=True, shared=True, kernel='exponential', matern_param=2.5)

results_1.loc[mod_name, "Time"] = np.mean(time_list)
results_1.loc[mod_name, "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc[mod_name, "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc[mod_name, "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc[mod_name, "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc[mod_name, "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc[mod_name, "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# 4. Linear Mixed Effects Model with Individual Gaussian Process
mod_name = 'Linear Mixed Effects Model with Independent Gaussian Process'
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, merf=False, linear=True, GP=True, shared=False, kernel='exponential', matern_param=2.5)

results_1.loc[mod_name, "Time"] = np.mean(time_list)
results_1.loc[mod_name, "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc[mod_name, "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc[mod_name, "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc[mod_name, "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc[mod_name, "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc[mod_name, "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# 5. Gradient tree-boosting including the grouping variable as a categorical variable ('Boosting_Cat')
datasets_cat, validation_datasets_cat = generate_datasets(n, m, p, n_datasets, n_valid, func='linear_func', 
                                                          shared_gp=True, random_state=60, include_cat_feature=True)

params = {'num_leaves': 10, 'max_depth': 4, 'min_data_in_leaf': 5}
time_list, RMSE_list1, RMSE_list2, _ = train_and_test(datasets_cat, n_datasets, merf=False, GPBoost_cat=True, linear=False, 
                                                      GP=False, num_boost_round=54, params=params)

results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "Time"] = np.mean(time_list)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["Gradient-boosted tree with group as categorical variable (no random effects)", "RMSE Interpolation (std)"] = np.std(RMSE_list2) 

  0%|          | 0/20 [00:00<?, ?it/s]

[GPBoost] [Info] Total Bins 536
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 6.851707
Error encountered
[GPBoost] [Info] Total Bins 535
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 6.440899
Error encountered
[GPBoost] [Info] Total Bins 535
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 6.243203
Error encountered
[GPBoost] [Info] Total Bins 535
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 6.810151
Error encountered
[GPBoost] [Info] Total Bins 536
[GPBoost] [Info] Number of data points in the train set: 300, number of used features: 6
[GPBoost] [Info] Start training from score 6.626740
Error encountered
[GPBoost] [Info] Total Bins 536
[GPBoost] [In

In [ ]:
# 6. GPBoost with Random Intercept
params = {'learning_rate': 0.5, 'max_depth': 1, 'min_data_in_leaf': 50, 
            'objective': 'regression_l2',
            'verbose': 0,
            'num_leaves': 2**10}
          
time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, num_boost_round=40, params=params, merf=False, linear=False, GP=False)
results_1.loc["GPBoost with Random Intercept", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Random Intercept", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Random Intercept", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Random Intercept", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Random Intercept", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Random Intercept", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Random Intercept", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# 7. GPBoost with Shared Gaussian Process
params = {'learning_rate': 0.5,
          'max_depth': 1,
          'min_data_in_leaf': 50,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, num_boost_round=209, params=params, merf=False, 
                                                            linear=False, GP=True, shared=True, kernel='exponential', matern_param=2.5)
results_1.loc["GPBoost with Shared Gaussian Process", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# 8. GPBoost with Independent Gaussian Process
params = {'learning_rate': 0.5,
          'max_depth': 1,
          'min_data_in_leaf': 50,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

time_list, RMSE_list1, RMSE_list2, F_list = train_and_test(datasets, n_datasets, num_boost_round=51, params=params,
                                         merf=False, linear=False, GP=True, shared=False, kernel='exponential', matern_param=2.5)
results_1.loc["GPBoost with Independent Gaussian Process", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Independent Gaussian Process", "RMSE_F (std)"] = np.std(F_list)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
results_1

,RMSE Interpolation (mean),RMSE Interpolation (std),RMSE Extrapolation (mean),RMSE Extrapolation (std),RMSE_F (mean),RMSE_F (std),Time
Linear Mixed Effects Model with no fixed features (random intercept),2.15775,0.150014,2.059487,0.136338,NaN,NaN,0.00826
Linear Mixed Effects Model with Random Intercept,1.912629,0.142163,1.786946,0.149038,1.981821,0.132306,0.006907
Linear Mixed Effects Model with Shared Gaussian Process,1.308962,0.073863,1.524178,0.184843,2.125227,0.200382,0.548429
Linear Mixed Effects Model with Independent Gaussian Process,1.736395,0.121791,1.81586,0.156605,1.971398,0.147458,0.062404
Gradient-boosted tree with group as categorical variable (no random effects),1.662303,0.147274,1.658576,0.17407,NaN,NaN,0.016125
GPBoost with Random Intercept,1.949604,0.145326,1.817582,0.146096,1.460892,0.146052,0.07527
GPBoost with Shared Gaussian Process,1.370582,0.068162,1.565544,0.186553,1.533803,0.158419,6.30808
GPBoost with Independent Gaussian Process,1.771432,0.119869,1.838275,0.158844,1.442436,0.162499,0.121362


In [ ]:
results_1.to_csv('small_data_sharedGP_linear.csv')